In [1]:
import sys
!{sys.executable} -m pip install traces

In [2]:
import pandas as pd
import numpy as np
import traces
from datetime import datetime

In [3]:
time_series = traces.TimeSeries()

In [4]:
df = pd.read_csv("ThermochronTracking Elephants Kruger 2007.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'],errors='ignore')
np.unique(df[['tag-local-identifier']].values)

array(['AM105', 'AM107', 'AM108', 'AM110', 'AM239', 'AM253', 'AM254',
       'AM255', 'AM306', 'AM307', 'AM308', 'AM91', 'AM93', 'AM99'],
      dtype=object)

In [6]:
df.head()

,event-id,visible,timestamp,location-long,location-lat,external-temperature,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name
0,9421351127,True,2007-08-13 00:30:00,31.87091,-24.81373,24.0,gps,Loxodonta africana,AM105,AM105,ThermochronTracking Elephants Kruger 2007
1,9421351128,True,2007-08-13 02:00:00,31.87399,-24.81483,23.0,gps,Loxodonta africana,AM105,AM105,ThermochronTracking Elephants Kruger 2007
2,9421351129,True,2007-08-13 03:31:00,31.87724,-24.81673,21.0,gps,Loxodonta africana,AM105,AM105,ThermochronTracking Elephants Kruger 2007
3,9421351130,True,2007-08-13 04:00:00,31.87822,-24.81569,21.0,gps,Loxodonta africana,AM105,AM105,ThermochronTracking Elephants Kruger 2007
4,9421351131,True,2007-08-13 06:00:00,31.89554,-24.79870,22.0,gps,Loxodonta africana,AM105,AM105,ThermochronTracking Elephants Kruger 2007


In [7]:
def small_dfs(dataframe, identifier, sampling_time, window):
    symbols = ['long', 'lat', 'temp']
    dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'],errors='ignore')
    raw = dataframe['individual-local-identifier'] == identifier
    as_frame = dataframe[raw]
    time = as_frame['timestamp']
    fg = {k:v for k,v in zip(time, as_frame['location-long'])}
    gdf = {k:v for k,v in zip(time, as_frame['location-lat'])}
    temp = {k:v for k,v in zip(time, as_frame['external-temperature'])}
        
    tsfg = pd.Series(fg, index=time)
    tsfg1 = traces.TimeSeries(tsfg)
    tsgdf = pd.Series(gdf, index=time)
    tsgdf1 = traces.TimeSeries(tsgdf)
    tstemp = pd.Series(temp, index=time)
    tstemp1 = traces.TimeSeries(tstemp)
    
    # 3600 = 1 Stunde, 10800 = 3 Stunden
    long = tsfg1.moving_average(sampling_period = sampling_time, window_size=32400,
                                placement= window,
                                pandas = True)
    lat = tsgdf1.moving_average(sampling_period = sampling_time,
                                placement= window,
                                pandas = True)
    temps = tstemp1.moving_average(sampling_period = sampling_time,
                                   placement= window,
                                   pandas = True)
    data = pd.concat([long, lat, temps], axis = 1)
    data.columns = symbols
    data['individual-local-identifier'] = identifier
    return data

In [8]:
def imputing_data(dataframe, sampling_time, window):
    time_series = traces.TimeSeries()
    identifiers = np.unique(dataframe['tag-local-identifier'].values)
    small_frame = []
    for i in identifiers:
        specific_frame = small_dfs(dataframe, i, sampling_time, window)
        small_frame.append(specific_frame)
    data_imputed = pd.concat(small_frame)
    return data_imputed

In [ ]:
data_18001 = imputing_data(df, 1800, "center")
data_18001.to_csv('imputed_data18001.csv')